## VAST Source Query & Analysis

This Notebook is for testing Radio lightcurve plotting for crossmatched VAST sources, printing png cutouts for said sources, as well as producing an Eta-V scatterplot for a family of sources to see the variability and statistical significance of them.

At the moment, with the current imports and code, plotting Radio lightcurves and producing png cutouts for a given source is working ok. Eta-V plots are functional, but take a long time to produce. A current goal is to produce a kernel density distribution in lieu of a scatterplot to save having to replot everything everytime. I want to see how a specific family of transients fares under the statistical analysis.

In [ ]:
import os
import requests
import pandas as pd
from io import StringIO
import numpy as np
import matplotlib.pyplot as plt
from vasttools.pipeline import Pipeline
from vasttools.query import Query
from bokeh.io import output_notebook
from bokeh.plotting import show
from astropy.coordinates import Angle, SkyCoord
from astropy import units as u
output_notebook()

In [ ]:
#just check and make sure you're in the right directory to load cmf.
#use os.chdir(pathname) if you need to change it
os.chdir('Project_VAST_FINK')
os.getcwd()
#alternatively, change the pathfile in the line below to where the .picklefile is

In [ ]:
cms = pd.read_pickle('Fink_2020_sources_matched_to_VAST_all_sources.pickle')

In [ ]:
pro.family_sort(cms)
print(cms.groupby('family').size().sort_values(ascending=False))

In [ ]:
#if we want analysis for a particular family, use this line
family_list=cms.query('family == "AGN"')

In [ ]:
#This will automatically find the base directory where all the runs are stored:
pipe=Pipeline()
#We can also load specific runs from the VAST pipeline:
my_run=pipe.load_run('tiles_corrected')

In [ ]:
#since matched id list is a list of strings, astype() converts them into intergers first
matched_ids=family_list['matched_id'].astype('int64').to_list()

In [ ]:
#locating sources in the 'tiles_corrected' run that have the same id
my_sources=my_run.sources.loc[matched_ids]
my_sources

## Eta - V Analysis:

In [ ]:
#this is the query string passed into the below function, which restricts the detected sources considered in the analysis. 
#Feel free to modify for different science goals.
my_query_string = (
    "n_measurements >= 3 "
    "& n_selavy >= 2 " # source finder ~ ML
    "& n_neighbour_dist > 1./60."
    "& 0.8 < avg_compactness < 1.4 "
    "& n_relations == 0"
    "& max_snr >= 5.0"
    "& v_peak > 0"
)

In [ ]:
#just remember to run output_notebook() from bokeh in order to plot the results using Bokeh 
eta_thresh, v_thresh, eta_v_candidates, plot = my_run.run_eta_v_analysis(
    1.0, 
    1.0, 
    query=my_query_string,
    plot_type='matplotlib' #comment this out if you want Bokeh
)
print('eta_thresh =',eta_thresh,', v_thresh = ',v_thresh)
print('The number of candidates is', len(eta_v_candidates))

In [ ]:
plot

At this point, we've plotted all the radio sources from the run with SNR>0 onto the plot. It takes a very long time for the analysis to finish. Dougal has suggested using a kernel density estimate that, while taking a long time, can be saved and replotted instantaneously. (kind of like saving a png)

Now, I want to plot the crossmatched FINK sources on this plot at the same time

In [ ]:
#This makes a list of all the matched_id's as intergers
listFinkmatched = cms.matched_id.astype(int).values.tolist()
listFinkmatched

#should be the same length as the original crossmatched source file
len(listFinkmatched)

In [ ]:
#this creates an arrray of sources from my_run that have the same ids as the catalogue, with the necessary eta and V information
#for the plot
sel=my_run.sources[my_run.sources.index.isin(listFinkmatched)]
print(len(sel))

You may notice by this point that the length of sel and listFinkmatched are not the same. This is because there are duplicate sources: unique FINK IDs assigned to the same VAST IDs. For the catalouge we looked at, there were approx. 60 sources that were duplicates, due to precision errors in identifying sources on FINKs part. This is looked at later in the notebook.

In [ ]:
#plot_2.axes[0] gives the axes for the plot, .scatter makes the scatterplot. since the axes are logarithmic (base 10), we
#have to take the log10 of each value of "eta_peak" and "v_peak" from sel.
plot.axes[0].scatter(np.log10(sel["eta_peak"]),np.log10(sel["v_peak"]), color='red',zorder=100)

In [ ]:
plot

Next, I want to try and take a subset of the plotted FINK sources under a particular family (like AGN) to see how the distribution looks. Note that you will have to clear all variables (which effectively means restarting the Kernel) to prevent multiple scatterplots being stacked on the same axis. Its recommended you save a given scatterplot to view later without having to rerun the whole notebook.

The below section can be run independantly of the above plotting if you just want a scatterplot of the given family sources:

In [ ]:
#simillar method as before, Im going to define the set of matched ids from the catalogue that belong to a given family
listFinkmatched_family=cms.query('family == "AGN"').matched_id.astype(int).values.tolist()

#again, the length of this should be equal to the number of family sources in the original catalogue
len(listFinkmatched_family)

In [ ]:
#same as before, define a new array of selected sources, this time for a specific family:
sel_family=my_run.sources[my_run.sources.index.isin(listFinkmatched_family)]

len(sel_family)

In [ ]:
#finally, plot them as before. This time the dots are green to differentiate them from the other plot:
plot.axes[0].scatter(np.log10(sel_family["eta_peak"]),np.log10(sel_family["v_peak"]), color='chocolate',zorder=100)

In [ ]:
plot

## Crossmatching Highly Variable Candidates With Catalogue

Our crossmatched catalogue only contains those sources that were detected by both ZTF and VAST. And thats great, but not all of those sources are either very variable or statistically significant or both. What we want to do know is crossmatch the crossmatch: find the sources within our VAST/FINK catalogue which are ALSO highly variable and statistically significant.

Thankfully, by performing our Eta-V Analysis, we already have our list of sources which fit the bill...

In [ ]:
#we already got the VAST info on our crossmatched sources through sel. we just need to filter for the highly variable
#sources based on the eta and v threshholds we calculated before:
candidate_sel = sel[(sel['eta_peak'] >= eta_thresh) & (sel['v_peak'] >= v_thresh)]

#getting the sel_candidate ids is squirrely, since they're the row INDEX of the dataframe. 
#passing eta_v_candidates['id'] will not work. The below code pulls out those index values as a string list:
candidate_ids=candidate_sel.index.values.astype('str').tolist()

#then we just check how many objects in cmf have an id that match the candidate ids
candidate_cmf=cms[cms['matched_id'].isin(candidate_ids)]

In [ ]:
#You can check to see the size of each family.
candidate_cms.groupby('family').size()

The ETA - V filtering thats been conducted above is also handled by the 'eta_v_analysis' function present in Projecttools.py, if you input the recorded eta_thresh and v_thresh that the initial analysis outputted.

## Duplicate Sources

Here, you can check if there happens to be any duplicate sources that might be present in your crossmatch catalogue

In [ ]:
#This looks for rows in cmf that have the same value for 'matched_id'. it dosent ignore the first instance of an id occuring
dup=cms[cms.duplicated(subset=['matched_id'],keep=False) == True]

In [ ]:
#displaying every duplicate row
pd.options.display.max_rows = None
dup.head(150)

In [ ]:
#here is the list of unique ids that are duplicates
pd.unique(dup['matched_id'])

In [ ]:
#here you can go through and pull out rows and their duplicates to see the difference
dup.query('matched_id == "3322564"')